In [4]:
import numpy as np
import random

# Sample data for product names and decisions
product_names = ["EPP", "Quick cash"]
decisions = ["Accepted", "Declined"]

# Generating a sample dataset of 1000 records (keep your original code here)
sample_dataset = []
for customer_id in range(1, 1001):
    interest_rate = round(random.uniform(4.5, 7.5), 2)
    product_name = random.choice(product_names)
    tenor = random.choice([6, 9, 12, 18, 24])
    decision = random.choice(decisions)
    
    record = {
        "customer_id": customer_id,
        "interest_rate": interest_rate,
        "product_name": product_name,
        "tenor": tenor,
        "decision": decision
    }
    sample_dataset.append(record)

# Q-learning parameters
learning_rate = 0.1
discount_factor = 0.9
num_episodes = 1000
epsilon = 0.1  # Exploration probability (epsilon-greedy)

# Initialize Q-table with zeros
num_actions = len(product_names) * len([6, 9, 12, 18, 24])
q_table = np.zeros((len(sample_dataset), num_actions))

# Create a mapping of product names, tenors, and interest rates to action index
action_to_idx = {}
idx = 0
for i, product in enumerate(product_names):
    for tenor in [6, 9, 12, 18, 24]:
        for rate in np.arange(4.5, 7.6, 0.1):  # Include interest rates from 4.5 to 7.5 with a step of 0.1
            action_to_idx[(product, tenor, round(rate, 2))] = idx
            idx += 1



# Create a mapping of decision to reward
reward_mapping = {
    "Accepted": 1.0,
    "Declined": 0.0
}

# Define state representation
def get_state_representation(customer_id):
    # For this example, the state representation will be the customer's historical data
    customer_data = [record for record in sample_dataset if record["customer_id"] == customer_id]
    state = [0] * num_actions
    for record in customer_data:
        product, tenor, rate = record["product_name"], record["tenor"], round(record["interest_rate"], 2)
        action_idx = action_to_idx[(product, tenor, rate)]
        state[action_idx] = rate
    return state



# Q-learning algorithm
# Q-learning algorithm
def q_learning(customer_id):
    state_idx = customer_id - 1  # State index is customer_id - 1 (since customer_id starts from 1)
    done = False

    for episode in range(num_episodes):
        if done:
            break

        # Get the current state representation
        state = get_state_representation(customer_id)

        if np.random.rand() < epsilon:
            # Explore: Select a random action
            action = np.random.choice(num_actions)
        else:
            # Exploit: Select the best action based on the Q-table
            action = np.argmax(q_table[state_idx, :])

        product_name, tenor = list(action_to_idx.keys())[action]
        interest_rate = state[action]

        # Present offers to the customer
        print("Offer Details:")
        print("Product Name:", product_name)
        print("Tenor:", tenor, "months")
        print("Interest Rate:", interest_rate, "%")

        # Customer responds with either "accept" or "decline"
        decision = input("Customer Response (accept/decline): ")
        decision = decision.capitalize()

        # Update the Q-table based on the customer response
        reward = reward_mapping.get(decision, 0.0)
        q_table[state_idx, action] = (1 - learning_rate) * q_table[state_idx, action] + \
                                     learning_rate * (reward + discount_factor * np.max(q_table[state_idx, :]))

        if decision == "Accept":
            done = True


# Example usage:
customer_id_to_offer = 1  # Replace this with the specific customer ID you want to offer recommendations for
q_learning(customer_id_to_offer)


KeyError: ('Quick cash', 24, 5.49)

In [6]:
import numpy as np

# Define the Q-learning parameters
num_actions = 6  # 2 products x 3 tenors
num_states = 2  # "Accepted" or "Declined"
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 1000

# Q-table initialization
Q = np.zeros((num_states, num_actions))

# Data in list format (Customer ID, Interest Rate, Product Name, Tenor, Decision)
data = [
    [1001, 2.5, "EPP", 60, "Accepted"],
    [1002, 4.5, "quick Cash", 12, "Declined"],
    [1004, 8.5, "EPP", 36, "Accepted"],
    [1006, 7.5, "EPP", 16, "Accepted"],
    [1002, 2.5, "quick Cash", 15, "Declined"],
    [1001, 6.5, "quick Cash", 40, "Declined"]
    # Add more records here
]

# Helper function to get the action index based on product name and tenor
def get_action_index(product_name, tenor):
    if product_name == "EPP":
        return tenor - 6
    elif product_name == "Quick Cash":
        return 3 + (tenor - 6)
    else:
        raise ValueError("Invalid product name or tenor.")

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state is "Accepted"
    for customer_data in data:
        customer_id, interest_rate, product_name, tenor, decision = customer_data
        action = get_action_index(product_name, tenor)
        reward = 1 if decision == "Accepted" else -1

        if np.random.uniform(0, 1) < epsilon:
            # Exploration: choose a random action
            next_action = np.random.randint(num_actions)
        else:
            # Exploitation: choose the action with the highest Q-value
            next_action = np.argmax(Q[state, :])

        # Update Q-value using Q-learning formula
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[state, :]) - Q[state, action])

        state = 0 if decision == "Accepted" else 1

# Function to recommend the next offer based on Q-tables
def recommend_offer(Q, customer_id):
    state = 0  # Initial state is "Accepted"
    offers = []

    while len(offers) < 3:
        # Choose the action with the highest Q-value
        action = np.argmax(Q[state, :])
        product_name = "EPP" if action < 3 else "Quick Cash"
        tenor = 6 + (action % 3)

        offers.append((product_name, tenor))

        # Ask the customer for response ("y" or "n")
        response = input(f"Customer ID {customer_id}: Offer - Product: {product_name}, Tenor: {tenor}. Accept? (y/n): ")

        if response.lower() == "y":
            return offers

        state = 0 if response.lower() == "n" else 1

    return offers

# Example usage:
customer_id = 1001  # Replace with the desired customer ID
recommended_offers = recommend_offer(Q, customer_id)
print("Recommended Offers:", recommended_offers)


IndexError: index 54 is out of bounds for axis 1 with size 6

In [7]:
import numpy as np

# Define the Q-learning parameters
num_actions = 18  # 3 interest rate bins x 2 products x 3 tenor bins
num_states = 2  # "Accepted" or "Declined"
num_interest_rate_bins = 3
num_tenor_bins = 3
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 1000

# Discretize interest rates and tenors into bins
interest_rate_bins = np.linspace(6.0, 10.0, num_interest_rate_bins + 1)
tenor_bins = np.linspace(6, 12, num_tenor_bins + 1)

# Q-table initialization
Q = np.zeros((num_states, num_actions))

# Data in list format (Customer ID, Interest Rate, Product Name, Tenor, Decision)
data = [
    [1001, 2.5, "EPP", 60, "Accepted"],
    [1002, 4.5, "quick Cash", 12, "Declined"],
    [1004, 8.5, "EPP", 36, "Accepted"],
    [1006, 7.5, "EPP", 16, "Accepted"],
    [1002, 2.5, "quick Cash", 15, "Declined"],
    [1001, 6.5, "quick Cash", 40, "Declined"]
    # Add more records here
]

# Helper function to get the action index based on product name, interest rate bin, and tenor bin
def get_action_index(product_name, interest_rate_bin, tenor_bin):
    product_offset = 0 if product_name == "EPP" else 9
    return product_offset + (interest_rate_bin * num_tenor_bins) + tenor_bin

# Helper function to discretize interest rates and tenors into bins
def discretize_value(value, bins):
    return np.digitize([value], bins)[0]

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state is "Accepted"
    for customer_data in data:
        customer_id, interest_rate, product_name, tenor, decision = customer_data
        interest_rate_bin = discretize_value(interest_rate, interest_rate_bins)
        tenor_bin = discretize_value(tenor, tenor_bins)
        action = get_action_index(product_name, interest_rate_bin, tenor_bin)
        reward = 1 if decision == "Accepted" else -1

        if np.random.uniform(0, 1) < epsilon:
            # Exploration: choose a random action
            next_action = np.random.randint(num_actions)
        else:
            # Exploitation: choose the action with the highest Q-value
            next_action = np.argmax(Q[state, :])

        # Update Q-value using Q-learning formula
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[state, :]) - Q[state, action])

        state = 0 if decision == "Accepted" else 1

# Function to recommend the next offer based on Q-tables
def recommend_offer(Q, customer_id):
    state = 0  # Initial state is "Accepted"
    offers = []

    while len(offers) < 3:
        # Choose the action with the highest Q-value
        action = np.argmax(Q[state, :])
        product_offset = 0 if action < 9 else 9
        interest_rate_bin, tenor_bin = divmod(action - product_offset, num_tenor_bins)
        interest_rate = (interest_rate_bins[interest_rate_bin] + interest_rate_bins[interest_rate_bin + 1]) / 2
        tenor = (tenor_bins[tenor_bin] + tenor_bins[tenor_bin + 1]) / 2
        product_name = "EPP" if product_offset == 0 else "Quick Cash"

        offers.append((product_name, interest_rate, tenor))

        # Ask the customer for response ("y" or "n")
        response = input(f"Customer ID {customer_id}: Offer - Product: {product_name}, Interest Rate: {interest_rate}, Tenor: {tenor}. Accept? (y/n): ")

        if response.lower() == "y":
            return offers

        state = 0 if response.lower() == "n" else 1

    return offers

# Example usage:
customer_id = 1001  # Replace with the desired customer ID
recommended_offers = recommend_offer(Q, customer_id)
print("Recommended Offers:", recommended_offers)


Customer ID 1001: Offer - Product: Quick Cash, Interest Rate: 6.666666666666666, Tenor: 9.0. Accept? (y/n): n
Customer ID 1001: Offer - Product: Quick Cash, Interest Rate: 6.666666666666666, Tenor: 9.0. Accept? (y/n): n
Customer ID 1001: Offer - Product: Quick Cash, Interest Rate: 6.666666666666666, Tenor: 9.0. Accept? (y/n): n
Recommended Offers: [('Quick Cash', 6.666666666666666, 9.0), ('Quick Cash', 6.666666666666666, 9.0), ('Quick Cash', 6.666666666666666, 9.0)]


In [10]:
import numpy as np

# Define the Q-learning parameters
num_actions = 18  # 3 interest rate bins x 2 products x 3 tenor bins
num_states = 2  # "Accepted" or "Declined"
num_interest_rate_bins = 3
num_tenor_bins = 3
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 1000

# Discretize interest rates and tenors into bins
interest_rate_bins = np.linspace(6.0, 10.0, num_interest_rate_bins + 1)
tenor_bins = np.linspace(6, 12, num_tenor_bins + 1)

# Q-table initialization
Q = np.zeros((num_states, num_actions))

# Data in list format (Customer ID, Interest Rate, Product Name, Tenor, Decision)
data = [
    [1001, 2.5, "EPP", 60, "Accepted"],
    [1002, 4.5, "quick Cash", 12, "Declined"],
    [1004, 8.5, "EPP", 36, "Accepted"],
    [1006, 7.5, "EPP", 16, "Accepted"],
    [1002, 2.5, "quick Cash", 15, "Declined"],
    [1001, 6.5, "quick Cash", 40, "Declined"]
    # Add more records here
]

# Helper function to get the action index based on product name, interest rate bin, and tenor bin
def get_action_index(product_name, interest_rate_bin, tenor_bin):
    product_offset = 0 if product_name == "EPP" else 9
    return product_offset + (interest_rate_bin * num_tenor_bins) + tenor_bin

# Helper function to discretize interest rates and tenors into bins
def discretize_value(value, bins):
    return np.digitize([value], bins)[0]

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state is "Accepted"
    for customer_data in data:
        customer_id, interest_rate, product_name, tenor, decision = customer_data
        interest_rate_bin = discretize_value(interest_rate, interest_rate_bins)
        tenor_bin = discretize_value(tenor, tenor_bins)
        action = get_action_index(product_name, interest_rate_bin, tenor_bin)
        reward = 1 if decision == "Accepted" else -1

        if np.random.uniform(0, 1) < epsilon:
            # Exploration: choose a random action
            next_action = np.random.randint(num_actions)
        else:
            # Exploitation: choose the action with the highest Q-value
            next_action = np.argmax(Q[state, :])

        # Update Q-value using Q-learning formula
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[state, :]) - Q[state, action])

        state = 0 if decision == "Accepted" else 1

# Function to recommend the next offer based on Q-tables
def recommend_offer(Q, customer_id):
    state = 0  # Initial state is "Accepted"
    offers = []

    while len(offers) < 3:
        # Choose the action with the highest Q-value for the current state
        action = np.argmax(Q[state, :])
        product_offset = 0 if action < 9 else 9
        interest_rate_bin, tenor_bin = divmod(action - product_offset, num_tenor_bins)
        interest_rate = (interest_rate_bins[interest_rate_bin] + interest_rate_bins[interest_rate_bin + 1]) / 2
        tenor = (tenor_bins[tenor_bin] + tenor_bins[tenor_bin + 1]) / 2
        product_name = "EPP" if product_offset == 0 else "Quick Cash"

        offers.append((product_name, interest_rate, tenor))

        # Ask the customer for response ("y" or "n")
        response = input(f"Customer ID {customer_id}: Offer - Product: {product_name}, Interest Rate: {interest_rate}, Tenor: {tenor}. Accept? (y/n): ")

        if response.lower() == "y":
            return offers

        state = 0 if response.lower() == "n" else 1

    return offers

# Example usage:
customer_id = 1001  # Replace with the desired customer ID
recommended_offers = recommend_offer(Q, customer_id)
print("Recommended Offers:", recommended_offers)


Customer ID 1001: Offer - Product: Quick Cash, Interest Rate: 6.666666666666666, Tenor: 9.0. Accept? (y/n): n
Customer ID 1001: Offer - Product: Quick Cash, Interest Rate: 6.666666666666666, Tenor: 9.0. Accept? (y/n): n
Customer ID 1001: Offer - Product: Quick Cash, Interest Rate: 6.666666666666666, Tenor: 9.0. Accept? (y/n): n
Recommended Offers: [('Quick Cash', 6.666666666666666, 9.0), ('Quick Cash', 6.666666666666666, 9.0), ('Quick Cash', 6.666666666666666, 9.0)]


In [22]:
import numpy as np

# Define the Q-learning parameters
num_actions = 18  # 3 interest rate bins x 2 products x 3 tenor bins
num_states = 2  # "Accepted" or "Declined"
num_interest_rate_bins = 3
num_tenor_bins = 3
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 1000

# Discretize interest rates and tenors into bins
interest_rate_bins = np.linspace(6.0, 10.0, num_interest_rate_bins + 1)
tenor_bins = np.linspace(6, 12, num_tenor_bins + 1)

# Q-table initialization
Q = np.zeros((num_states, num_actions))

# Data in list format (Customer ID, Interest Rate, Product Name, Tenor, Decision)
data = [
    [1001, 2.5, "EPP", 60, "Accepted"],
    [1002, 4.5, "quick Cash", 12, "Declined"],
    [1004, 8.5, "EPP", 36, "Accepted"],
    [1006, 7.5, "EPP", 16, "Accepted"],
    [1002, 2.5, "quick Cash", 15, "Declined"],
    [1001, 6.5, "quick Cash", 40, "Declined"]
    # Add more records here
]

# Helper function to get the action index based on product name, interest rate bin, and tenor bin
def get_action_index(product_name, interest_rate_bin, tenor_bin):
    product_offset = 0 if product_name == "EPP" else 9
    return product_offset + (interest_rate_bin * num_tenor_bins) + tenor_bin

# Helper function to discretize interest rates and tenors into bins
def discretize_value(value, bins):
    return np.digitize([value], bins)[0]

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state is "Accepted"
    for customer_data in data:
        customer_id, interest_rate, product_name, tenor, decision = customer_data
        interest_rate_bin = discretize_value(interest_rate, interest_rate_bins)
        tenor_bin = discretize_value(tenor, tenor_bins)
        action = get_action_index(product_name, interest_rate_bin, tenor_bin)
        reward = 1 if decision == "Accepted" else -1

        if np.random.uniform(0, 1) < epsilon:
            # Exploration: choose a random action
            next_action = np.random.randint(num_actions)
        else:
            # Exploitation: choose the action with the highest Q-value
            next_action = np.argmax(Q[state, :])

        # Update Q-value using Q-learning formula
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[state, :]) - Q[state, action])

        state = 0 if decision == "Accepted" else 1

# ... (Rest of the code remains the same)

# ... (Rest of the code remains the same)

# Function to recommend the next offer based on Q-tables
def recommend_offer(Q, customer_id):
    state = 0  # Initial state is "Accepted"
    offers = []
    offered_products = set()
    action = None  # Initialize action to None
    interest_rate = (interest_rate_bins[0] + interest_rate_bins[-1]) / 2  # Initialize interest_rate to the midpoint of bins
    tenor = (tenor_bins[0] + tenor_bins[-1]) / 2  # Initialize tenor to the midpoint of bins

    while len(offers) < 3:
        if np.random.uniform(0, 1) < epsilon_product or len(offers) == 0:
            # Random product selection during exploration or for the first offer
            product_name = "EPP" if np.random.uniform(0, 1) < 0.5 else "Quick Cash"
        else:
            # Choose the action with the highest Q-value for the current state, excluding offered products
            valid_actions = [a for a in range(num_actions) if a not in offered_products]
            if len(valid_actions) == 0:
                # All products are offered at least once, so choose randomly
                action = np.random.randint(num_actions)
            else:
                action = valid_actions[np.argmax(Q[state, valid_actions])]

            product_offset = 0 if action < 9 else 9
            interest_rate_bin, tenor_bin = divmod(action - product_offset, num_tenor_bins)
            interest_rate = (interest_rate_bins[interest_rate_bin] + interest_rate_bins[interest_rate_bin + 1]) / 2
            tenor = (tenor_bins[tenor_bin] + tenor_bins[tenor_bin + 1]) / 2
            product_name = "EPP" if product_offset == 0 else "Quick Cash"

        offers.append((product_name, interest_rate, tenor))
        offered_products.add(action)

        # Ask the customer for response ("y" or "n")
        response = input(f"Customer ID {customer_id}: Offer - Product: {product_name}, Interest Rate: {interest_rate}, Tenor: {tenor}. Accept? (y/n): ")

        if response.lower() == "y":
            return offers

        state = 0 if response.lower() == "n" else 1

    return offers

# Example usage:
customer_id = 1005  # Replace with the desired customer ID
recommended_offers = recommend_offer(Q, customer_id)
print("Recommended Offers:", recommended_offers)


Customer ID 1005: Offer - Product: Quick Cash, Interest Rate: 8.0, Tenor: 9.0. Accept? (y/n): n
Customer ID 1005: Offer - Product: Quick Cash, Interest Rate: 6.666666666666666, Tenor: 9.0. Accept? (y/n): n
Customer ID 1005: Offer - Product: EPP, Interest Rate: 8.0, Tenor: 9.0. Accept? (y/n): n
Recommended Offers: [('Quick Cash', 8.0, 9.0), ('Quick Cash', 6.666666666666666, 9.0), ('EPP', 8.0, 9.0)]


In [26]:
import numpy as np

# Define the Q-learning parameters
num_actions = 18  # 3 interest rate bins x 2 products x 3 tenor bins
num_states = 2  # "Accepted" or "Declined"
num_interest_rate_bins = 3
num_tenor_bins = 3
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 1000

# Discretize interest rates and tenors into bins
interest_rate_bins = np.linspace(6.0, 10.0, num_interest_rate_bins + 1)
tenor_bins = np.linspace(6, 12, num_tenor_bins + 1)

# Q-table initialization
Q = np.zeros((num_states, num_actions))

# Data in list format (Customer ID, Interest Rate, Product Name, Tenor, Decision)
data = [
    [1001, 2.5, "EPP", 60, "Accepted"],
    [1002, 4.5, "quick Cash", 12, "Declined"],
    [1004, 8.5, "EPP", 36, "Accepted"],
    [1006, 7.5, "EPP", 16, "Accepted"],
    [1002, 2.5, "quick Cash", 15, "Declined"],
    [1001, 6.5, "quick Cash", 40, "Declined"]
    # Add more records here
]

# Helper function to get the action index based on product name, interest rate bin, and tenor bin
def get_action_index(product_name, interest_rate_bin, tenor_bin):
    product_offset = 0 if product_name == "EPP" else 9
    return product_offset + (interest_rate_bin * num_tenor_bins) + tenor_bin

# Helper function to discretize interest rates and tenors into bins
def discretize_value(value, bins):
    return np.digitize([value], bins)[0]

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state is "Accepted"
    for customer_data in data:
        customer_id, interest_rate, product_name, tenor, decision = customer_data
        interest_rate_bin = discretize_value(interest_rate, interest_rate_bins)
        tenor_bin = discretize_value(tenor, tenor_bins)
        action = get_action_index(product_name, interest_rate_bin, tenor_bin)
        reward = 1 if decision == "Accepted" else -1

        if np.random.uniform(0, 1) < epsilon:
            # Exploration: choose a random action
            next_action = np.random.randint(num_actions)
        else:
            # Exploitation: choose the action with the highest Q-value
            next_action = np.argmax(Q[state, :])

        # Update Q-value using Q-learning formula
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[state, :]) - Q[state, action])

        state = 0 if decision == "Accepted" else 1

# ... (Rest of the code remains the same)

# ... (Rest of the code remains the same)

# Define the default offer (change as per your requirements)
default_product = "EPP"
default_interest_rate = 8.0
default_tenor = 12.0

# ... (Rest of the code remains the same)

# Define the default offer (change as per your requirements)
default_product = "EPP"
default_interest_rate = 8.0
default_tenor = 12.0

# Function to recommend the next offer based on Q-tables
def recommend_offer(Q, customer_id):
    state = 0  # Initial state is "Accepted"
    offers = []
    offered_products = set()
    action = None  # Initialize action to None
    interest_rate = (interest_rate_bins[0] + interest_rate_bins[-1]) / 2  # Initialize interest_rate to the midpoint of bins
    tenor = (tenor_bins[0] + tenor_bins[-1]) / 2  # Initialize tenor to the midpoint of bins

    while len(offers) < 3:
        if len(offers) >= threshold:
            # After the threshold number of offers, offer the default product
            product_name = default_product
            interest_rate = default_interest_rate
            tenor = default_tenor
        elif np.random.uniform(0, 1) < epsilon_product or len(offers) == 0:
            # Random product selection during exploration or for the first offer
            product_name = "EPP" if np.random.uniform(0, 1) < 0.5 else "Quick Cash"
        else:
            # Choose the action with the highest Q-value for the current state, excluding offered products
            valid_actions = [a for a in range(num_actions) if a not in offered_products]
            if len(valid_actions) == 0:
                # All products are offered at least once, so choose randomly
                action = np.random.randint(num_actions)
            else:
                action = valid_actions[np.argmax(Q[state, valid_actions])]

            product_offset = 0 if action < 9 else 9
            interest_rate_bin, tenor_bin = divmod(action - product_offset, num_tenor_bins)
            interest_rate = (interest_rate_bins[interest_rate_bin] + interest_rate_bins[interest_rate_bin + 1]) / 2
            tenor = (tenor_bins[tenor_bin] + tenor_bins[tenor_bin + 1]) / 2
            product_name = "EPP" if product_offset == 0 else "Quick Cash"

        offers.append((product_name, interest_rate, tenor))
        offered_products.add(action)

        # Ask the customer for response ("y" or "n")
        response = input(f"Customer ID {customer_id}: Offer - Product: {product_name}, Interest Rate: {interest_rate}, Tenor: {tenor}. Accept? (y/n): ")

        if response.lower() == "y":
            return offers

        state = 0 if response.lower() == "n" else 1

    return offers

# Example usage:
customer_id = 1005  # Replace with the desired customer ID
threshold = 3  # Replace with the desired threshold value
recommended_offers = recommend_offer(Q, customer_id)
print("Recommended Offers:", recommended_offers)


Customer ID 1005: Offer - Product: Quick Cash, Interest Rate: 8.0, Tenor: 9.0. Accept? (y/n): n
Customer ID 1005: Offer - Product: EPP, Interest Rate: 8.0, Tenor: 9.0. Accept? (y/n): n
Customer ID 1005: Offer - Product: Quick Cash, Interest Rate: 6.666666666666666, Tenor: 9.0. Accept? (y/n): n
Recommended Offers: [('Quick Cash', 8.0, 9.0), ('EPP', 8.0, 9.0), ('Quick Cash', 6.666666666666666, 9.0)]


In [30]:
import numpy as np

# Define the Q-learning parameters
num_actions = 18  # 3 interest rate bins x 2 products x 3 tenor bins
num_states = 2  # "Accepted" or "Declined"
num_interest_rate_bins = 3
num_tenor_bins = 3
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 1000

# Discretize interest rates and tenors into bins
interest_rate_bins = np.linspace(6.0, 10.0, num_interest_rate_bins + 1)
tenor_bins = np.linspace(6, 12, num_tenor_bins + 1)

# Q-table initialization
Q = np.zeros((num_states, num_actions))

# Data in list format (Customer ID, Interest Rate, Product Name, Tenor, Decision)
data = [
    [1001, 2.5, "EPP", 60, "Accepted"],
    [1002, 4.5, "quick Cash", 12, "Declined"],
    [1004, 8.5, "EPP", 36, "Accepted"],
    [1006, 7.5, "EPP", 16, "Accepted"],
    [1002, 2.5, "quick Cash", 15, "Declined"],
    [1001, 6.5, "quick Cash", 40, "Declined"]
    # Add more records here
]

# Helper function to get the action index based on product name, interest rate bin, and tenor bin
def get_action_index(product_name, interest_rate_bin, tenor_bin):
    product_offset = 0 if product_name == "EPP" else 9
    return product_offset + (interest_rate_bin * num_tenor_bins) + tenor_bin

# Helper function to discretize interest rates and tenors into bins
def discretize_value(value, bins):
    return np.digitize([value], bins)[0]

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state is "Accepted"
    for customer_data in data:
        customer_id, interest_rate, product_name, tenor, decision = customer_data
        interest_rate_bin = discretize_value(interest_rate, interest_rate_bins)
        tenor_bin = discretize_value(tenor, tenor_bins)
        action = get_action_index(product_name, interest_rate_bin, tenor_bin)
        reward = 1 if decision == "Accepted" else -1

        if np.random.uniform(0, 1) < epsilon:
            # Exploration: choose a random action
            next_action = np.random.randint(num_actions)
        else:
            # Exploitation: choose the action with the highest Q-value
            next_action = np.argmax(Q[state, :])

        # Update Q-value using Q-learning formula
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[state, :]) - Q[state, action])

        state = 0 if decision == "Accepted" else 1

# ... (Rest of the code remains the same)

# ... (Rest of the code remains the same)

# Define the default offer (change as per your requirements)
default_product = "EPP"
default_interest_rate = 8.0
default_tenor = 12.0

# ... (Rest of the code remains the same)

# ... (Rest of the code remains the same)

# Define the default offer (change as per your requirements)
default_product = "EPP"
default_interest_rate = 8.0
default_tenor = 12.0

# Function to recommend the next offer based on Q-tables
def recommend_offer(Q, customer_id):
    state = 0  # Initial state is "Accepted"
    offers = []
    offered_products = set()
    action = None  # Initialize action to None
    interest_rate = (interest_rate_bins[0] + interest_rate_bins[-1]) / 2  # Initialize interest_rate to the midpoint of bins
    tenor = (tenor_bins[0] + tenor_bins[-1]) / 2  # Initialize tenor to the midpoint of bins

    while len(offers) < 3:
        if len(offers) >= threshold:
            # After the threshold number of offers, offer the default product
            return [(default_product, default_interest_rate, default_tenor)]

        if np.random.uniform(0, 1) < epsilon_product or len(offers) == 0:
            # Random product selection during exploration or for the first offer
            product_name = "EPP" if np.random.uniform(0, 1) < 0.5 else "Quick Cash"
        else:
            # Choose the action with the highest Q-value for the current state, excluding offered products
            valid_actions = [a for a in range(num_actions) if a not in offered_products]
            if len(valid_actions) == 0:
                # All products are offered at least once, so choose randomly
                action = np.random.randint(num_actions)
            else:
                action = valid_actions[np.argmax(Q[state, valid_actions])]

            product_offset = 0 if action < 9 else 9
            interest_rate_bin, tenor_bin = divmod(action - product_offset, num_tenor_bins)
            interest_rate = (interest_rate_bins[interest_rate_bin] + interest_rate_bins[interest_rate_bin + 1]) / 2
            tenor = (tenor_bins[tenor_bin] + tenor_bins[tenor_bin + 1]) / 2
            product_name = "EPP" if product_offset == 0 else "Quick Cash"

        offers.append((product_name, interest_rate, tenor))
        offered_products.add(action)

        # Ask the customer for response ("y" or "n")
        response = input(f"Customer ID {customer_id}: Offer - Product: {product_name}, Interest Rate: {interest_rate}, Tenor: {tenor}. Accept? (y/n): ")

        if response.lower() == "y":
            return offers

        state = 0 if response.lower() == "n" else 1

    return offers

# Example usage:
customer_id = 1005  # Replace with the desired customer ID
threshold = 3  # Replace with the desired threshold value
recommended_offers = recommend_offer(Q, customer_id)

# If the threshold is reached, show only the default offer
if len(recommended_offers) >= threshold:
    print(f"Recommended Default Offers: {[(default_product, default_interest_rate, default_tenor)]}")
else:
    print("Recommended Offers:", recommended_offers)



Customer ID 1005: Offer - Product: EPP, Interest Rate: 8.0, Tenor: 9.0. Accept? (y/n): n
Customer ID 1005: Offer - Product: Quick Cash, Interest Rate: 6.666666666666666, Tenor: 9.0. Accept? (y/n): n
Customer ID 1005: Offer - Product: EPP, Interest Rate: 8.0, Tenor: 9.0. Accept? (y/n): n
Recommended Default Offers: [('EPP', 8.0, 12.0)]


In [ ]:
import numpy as np

# Define the Q-learning parameters
num_actions = 18  # 3 interest rate bins x 2 products x 3 tenor bins
num_states = 2  # "Accepted" or "Declined"
num_interest_rate_bins = 3
num_tenor_bins = 3
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 1000

# Discretize interest rates and tenors into bins
interest_rate_bins = np.linspace(6.0, 10.0, num_interest_rate_bins + 1)
tenor_bins = np.linspace(6, 12, num_tenor_bins + 1)

# Q-table initialization
Q = np.zeros((num_states, num_actions))

# Data in list format (Customer ID, Interest Rate, Product Name, Tenor, Decision)
data = [
    [1001, 2.5, "EPP", 60, "Accepted"],
    [1002, 4.5, "quick Cash", 12, "Declined"],
    [1004, 8.5, "EPP", 36, "Accepted"],
    [1006, 7.5, "EPP", 16, "Accepted"],
    [1002, 2.5, "quick Cash", 15, "Declined"],
    [1001, 6.5, "quick Cash", 40, "Declined"]
    # Add more records here
]

# Helper function to get the action index based on product name, interest rate bin, and tenor bin
def get_action_index(product_name, interest_rate_bin, tenor_bin):
    product_offset = 0 if product_name == "EPP" else 9
    return product_offset + (interest_rate_bin * num_tenor_bins) + tenor_bin

# Helper function to discretize interest rates and tenors into bins
def discretize_value(value, bins):
    return np.digitize([value], bins)[0]

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state is "Accepted"
    for customer_data in data:
        customer_id, interest_rate, product_name, tenor, decision = customer_data
        interest_rate_bin = discretize_value(interest_rate, interest_rate_bins)
        tenor_bin = discretize_value(tenor, tenor_bins)
        action = get_action_index(product_name, interest_rate_bin, tenor_bin)
        reward = 1 if decision == "Accepted" else -1

        if np.random.uniform(0, 1) < epsilon:
            # Exploration: choose a random action
            next_action = np.random.randint(num_actions)
        else:
            # Exploitation: choose the action with the highest Q-value
            next_action = np.argmax(Q[state, :])

        # Update Q-value using Q-learning formula
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[state, :]) - Q[state, action])

        state = 0 if decision == "Accepted" else 1


# Define the default offer (change as per your requirements)
default_product = "EPP"
default_interest_rate = 8.0
default_tenor = 12.0



# Function to recommend the next offer based on Q-tables
def recommend_offer(Q, customer_id):
    state = 0  # Initial state is "Accepted"
    offers = []
    offered_products = set()
    action = None  # Initialize action to None
    interest_rate = (interest_rate_bins[0] + interest_rate_bins[-1]) / 2  # Initialize interest_rate to the midpoint of bins
    tenor = (tenor_bins[0] + tenor_bins[-1]) / 2  # Initialize tenor to the midpoint of bins

    while len(offers) < 3:
        if len(offers) >= threshold:
            # After the threshold number of offers, offer the default product
            return [(default_product, default_interest_rate, default_tenor)]

        if np.random.uniform(0, 1) < epsilon_product or len(offers) == 0:
            # Random product selection during exploration or for the first offer
            product_name = "EPP" if np.random.uniform(0, 1) < 0.5 else "Quick Cash"
        else:
            # Choose the action with the highest Q-value for the current state, excluding offered products
            valid_actions = [a for a in range(num_actions) if a not in offered_products]
            if len(valid_actions) == 0:
                # All products are offered at least once, so choose randomly
                action = np.random.randint(num_actions)
            else:
                action = valid_actions[np.argmax(Q[state, valid_actions])]

            product_offset = 0 if action < 9 else 9
            interest_rate_bin, tenor_bin = divmod(action - product_offset, num_tenor_bins)
            interest_rate = (interest_rate_bins[interest_rate_bin] + interest_rate_bins[interest_rate_bin + 1]) / 2
            tenor = (tenor_bins[tenor_bin] + tenor_bins[tenor_bin + 1]) / 2
            product_name = "EPP" if product_offset == 0 else "Quick Cash"

        offers.append((product_name, interest_rate, tenor))
        offered_products.add(action)

        # Ask the customer for response ("y" or "n")
        response = input(f"Customer ID {customer_id}: Offer - Product: {product_name}, Interest Rate: {interest_rate}, Tenor: {tenor}. Accept? (y/n): ")

        if response.lower() == "y":
            return offers

        state = 0 if response.lower() == "n" else 1

    return offers

# Example usage:
customer_id = 1005  # Replace with the desired customer ID
threshold = 3  # Replace with the desired threshold value
recommended_offers = recommend_offer(Q, customer_id)

# If the threshold is reached, show only the default offer
if len(recommended_offers) >= threshold:
    print(f"Recommended Default Offers: {[(default_product, default_interest_rate, default_tenor)]}")
else:
    print("Recommended Offers:", recommended_offers)



In [ ]:
import pandas as pd

data = [
    [1001, 2.5, "EPP", 60, "Accepted"],
    [1002, 4.5, "quick Cash", 12, "Declined"],
    [1004, 8.5, "EPP", 36, "Accepted"],
    [1006, 7.5, "EPP", 16, "Accepted"],
    [1002, 2.5, "quick Cash", 15, "Declined"],
    [1001, 6.5, "quick Cash", 40, "Declined"]
    # Add more records here
]

# Define the column names
columns = ["ID", "Score", "Type", "Duration", "Status"]

# Create the DataFrame
df = pd.DataFrame(data, columns=columns)

# Filter "Accepted" and "Declined" records separately
accepted_df = df[df['Status'] == 'Accepted']
declined_df = df[df['Status'] == 'Declined']

# Sample "Declined" records with a 1:2 ratio
sampled_declined_df = declined_df.sample(n=len(accepted_df) * 2, replace=True)

# Concatenate the sampled "Accepted" and "Declined" DataFrames
final_df = pd.concat([accepted_df, sampled_declined_df])

# Reset the index of the final DataFrame
final_df = final_df.reset_index(drop=True)

# Convert the final DataFrame to a list of lists
final_data = final_df.values.tolist()

# Print the sampled data
print(final_data)
